## Tests of connection to the Neo4j database, and basic NeoAccess library operations
#### (`debug` mode OFF)

In [1]:
import set_path    # Add the project's home directory to sys.path

Added 'D:\Docs\- MY CODE\Brain Annex\BA-Win7' to sys.path


In [2]:
import os
import sys
import getpass

from BrainAnnex.modules.neo_access.neo_access import NeoAccess

In [3]:
print("To create a database connection, enter the host IP, but leave out the port number: (EXAMPLES:  bolt://1.2.3.4  OR  neo4j://localhost )\n")

host = input("Enter host IP WITHOUT the port number.  EXAMPLE: bolt://123.456.789.012")
host += ":7687"    # EXAMPLE of host value:  "bolt://123.456.789.012:7687"

password = getpass.getpass("Enter the database password:")

print(f"\n=> Will be using: host='{host}', username='neo4j', password=**********")

To create a database connection, enter the host IP, but leave out the port number: (EXAMPLES:  bolt://1.2.3.4  OR  neo4j://localhost )



Enter host IP WITHOUT the port number.  EXAMPLE: bolt://123.456.789.012 bolt://155.248.202.124
Enter the database password: ········



=> Will be using: host='bolt://155.248.202.124:7687', username='neo4j', password=**********


In [4]:
db = NeoAccess(host=host,
                credentials=("neo4j", password), debug=False)

Attempting to connect to Neo4j host 'bolt://155.248.202.124:7687', with username 'neo4j'


In [5]:
print("Version of the Neo4j driver: ", db.version())

Version of the Neo4j driver:  4.3.9


In [6]:
db.empty_dbase()       # WARNING: USE WITH CAUTION!!!

In [7]:
neo_car = db.create_node("Car", {'color': 'white', 'make': 'Toyota'})
neo_person = db.create_node("Person", {'name': 'Julian'})

In [8]:
number_added = db.add_links(neo_car, neo_person, rel_name="OWNED_BY")

number_added

1

In [9]:
# Retrieve the car nodes
db.get_nodes(neo_car)

[{'color': 'white', 'make': 'Toyota'}]

In [10]:
# Retrieve a single property of the car node (for situation when only 1 node is present)
db.get_nodes(neo_car, single_cell="color")

'white'

In [11]:
# How many owners does the car have?
db.count_links(neo_car, rel_name="OWNED_BY", rel_dir="OUT") 

1

In [12]:
# Look up information about the car owner(s)
db.follow_links(neo_car, rel_name="OWNED_BY", rel_dir="OUT") 

[{'name': 'Julian'}]